In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter

In [ ]:
os.chdir('/home/cwlin/explainable_credit/data/raw_data/')

In [2]:
company_mapping_file = "Company_Mapping.csv"
us_firms_specific_dir = "US_Firms_Specific"

df_company_mapping = pd.read_csv(company_mapping_file)
dfs = []
for i in range(1, 5):
    file_path = f"{us_firms_specific_dir}/US_Firms_Specific_{i}.csv"
    dfs.append(pd.read_csv(file_path))
df_us_firms_specific = pd.concat(dfs, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Company_Mapping.csv'

In [14]:
df_company_mapping["CompanyID"] = df_company_mapping["CompanyID"].astype(str)
df_us_firms_specific["Company Mapping No."] = df_us_firms_specific["Company Mapping No."].astype(str)

# Only retain the following columns /
filter_columns = ['Company Mapping No.', 'Year', 'Month']
df_us_firms_specific = df_us_firms_specific[filter_columns]

# Retrieve the first 5 digits of each company number
# Retrieve the last 3 digits of each company number
# df_us_firms_specific["First_Five_Digits"] = df_us_firms_specific["Company Mapping No."].str[:5]
# df_us_firms_specific["Last_Three_Digits"] = df_us_firms_specific["Company Mapping No."].str[5:]

In [15]:
df_company_mapping.head()

,CompanyID,Company Name,Firm Type,Unnamed: 3,Unnamed: 4
0,22043,CHF Solutions Inc,0,NaN,NaN
1,26978,AAR Corp,0,NaN,0 = Financial
2,26979,Abbott Laboratories,0,NaN,1 = Nonfinancial
3,26980,Acme Electric Corp,0,NaN,NaN
4,26981,Acuson Corp,0,NaN,NaN


In [16]:
df_us_firms_specific.head()

,Company Mapping No.,Year,Month
0,22043000,2015,8
1,22043000,2015,9
2,22043000,2015,10
3,22043000,2015,11
4,22043000,2015,12


# Find the CompanyID mismatch

## check the number of digits of company id

In [17]:
unique_CompanyID_in_mapping = df_company_mapping["CompanyID"].unique()
print("Total number of unique company id:", len(unique_CompanyID_in_mapping))

# Count the occurrences of each unique length
length_counts = Counter(len(s) for s in unique_CompanyID_in_mapping)

print(length_counts)

Total number of unique company id: 15639
Counter({5: 12745, 6: 2894})


In [18]:
unique_CompanyNo_in_specific = df_us_firms_specific["Company Mapping No."].unique()
print("Total number of unique company number:", len(unique_CompanyNo_in_specific))

# Count the occurrences of each unique length
length_counts = Counter(len(s) for s in unique_CompanyNo_in_specific)

print(length_counts)

Total number of unique company number: 18066
Counter({8: 15044, 9: 3022})


## Filter the CompanyID that is not a prefix of CompanyNo

In [19]:
# Filter CompanyIDs that are not prefix of any CompanyNo
filtered_CompanyID = [
    company_id for company_id in unique_CompanyID_in_mapping
    if all(not company_no.startswith(company_id) for company_no in unique_CompanyNo_in_specific)
]
print(filtered_CompanyID) # 所有company_id都是某個company_no的prefix

[]


# See CompanyID as prefix and the rest of digits in CompanyNo as suffix

In [9]:
# Initialize an empty dictionary to hold the company_id and suffix relationship
company_id_suffix_dict = {}

# Iterate over each CompanyID
for company_id in unique_CompanyID_in_mapping:
    # Initialize an empty list to hold suffixes for the current CompanyID
    suffix_list = []
    
    # Iterate over each CompanyNo
    for company_no in unique_CompanyNo_in_specific:
        # Check if the CompanyNo starts with the current CompanyID
        if company_no.startswith(company_id):
            # Extract the suffix from CompanyNo and append to suffix_list
            suffix = company_no[len(company_id):]
            suffix_list.append(suffix)
    
    # If suffix_list for the current CompanyID is not empty, add to the dictionary
    if suffix_list:
        company_id_suffix_dict[company_id] = suffix_list

# Print or use the resulting dictionary
print(company_id_suffix_dict)


{'22043': ['000', '001'], '26978': ['000', '001', '002', '003', '004'], '26979': ['000', '001'], '26980': ['000'], '26981': ['000'], '26982': ['000'], '26983': ['000', '001'], '26984': ['000'], '26985': ['000'], '26986': ['000'], '26989': ['000'], '26990': ['000'], '26991': ['000'], '26992': ['000'], '26994': ['000'], '26995': ['000'], '26996': ['000'], '26998': ['000'], '26999': ['000'], '27000': ['000'], '27001': ['000'], '27002': ['000'], '27003': ['000'], '27004': ['000'], '27005': ['000'], '27006': ['000', '001'], '27007': ['000'], '27008': ['000'], '27009': ['000'], '27010': ['000'], '27011': ['000'], '27012': ['000', '001'], '27013': ['000'], '27014': ['000'], '27016': ['000', '001'], '27017': ['000'], '27018': ['000', '001'], '27019': ['000'], '27020': ['000'], '27021': ['000'], '27022': ['000'], '27024': ['000'], '27025': ['000'], '27026': ['000', '001'], '27027': ['000'], '27028': ['000'], '27029': ['000', '001'], '27030': ['000'], '27031': ['000'], '27032': ['000'], '27033':

In [62]:
multi_suffix = 0
for key, list in company_id_suffix_dict.items():
    if len(list) > 1:
        multi_suffix += 1
print("# of prefix with multiple suffix:", multi_suffix)
print("# of total prefix:", len(company_id_suffix_dict.keys()))

# of prefix with multiple suffix: 2083
# of total prefix: 15639


In [12]:
# check if number of total suffix equals the number of total CompanyNo
total_no_cnt = 0
for list in company_id_suffix_dict.items():
    total_no_cnt += len(list)

print("Number of suffix:", total_no_cnt)
print("Total number of unique company number:", len(unique_CompanyNo_in_specific))
if total_no_cnt == len(unique_CompanyNo_in_specific):
    print("# of suffix equals # of CompanyNo")
else:
    print("not equal")

Number of suffix: 31278
Total number of unique company number: 18066
not equal


In [11]:
total_no_cnt = 0
for list in company_id_suffix_dict.keys():
    total_no_cnt += 1
print(total_no_cnt)

15639


In [20]:
df_us_firms_specific.head()

,Company Mapping No.,Year,Month
0,22043000,2015,8
1,22043000,2015,9
2,22043000,2015,10
3,22043000,2015,11
4,22043000,2015,12


In [54]:
result_df = pd.DataFrame(columns=["prefix", "suffix", "start_year", "start_month", "end_year", "end_month"])
company_ids_not_prefix_of_companyNo = []

for company_id in unique_CompanyID_in_mapping:
    # print(company_id)
    filtered_df = df_us_firms_specific[df_us_firms_specific["Company Mapping No."].str.startswith(company_id)]
    
    if filtered_df.empty:
        company_ids_not_prefix_of_companyNo.append(company_id)
        continue
    
    for suffix in filtered_df["Company Mapping No."].str[len(company_id):].unique():
        # print(suffix)
        # For each suffix, find the row with the minimum year and month
        min_year_month_row = filtered_df[filtered_df["Company Mapping No."].str.endswith(suffix)].nsmallest(1, ["Year", "Month"])
        max_year_month_row = filtered_df[filtered_df["Company Mapping No."].str.endswith(suffix)].nlargest(1, ["Year", "Month"])
        
        # Extract the values and append to the result DataFrame
        new_row = pd.DataFrame({
            "prefix": [company_id], 
            "suffix": [suffix], 
            "start_year": [min_year_month_row["Year"].values[0]], 
            "start_month": [min_year_month_row["Month"].values[0]], 
            "end_year": [max_year_month_row["Year"].values[0]], 
            "end_month": [max_year_month_row["Month"].values[0]]
        })
        result_df = pd.concat([result_df, new_row], ignore_index=True)

result_df.reset_index(drop=True, inplace=True)

print("# of company_id that is not a prefix of a CompanyNo:", len(company_ids_not_prefix_of_companyNo))

# of company_id that is not a prefix of a CompanyNo: 0


In [59]:
result_df

,prefix,suffix,start_year,start_month,end_year,end_month
0,22043,000,2015,8,2017,12
1,22043,001,2004,9,2013,3
2,26978,000,2009,5,2017,12
3,26978,001,2006,5,2009,4
4,26978,002,2005,2,2006,4
...,...,...,...,...,...,...
18061,198308,000,2017,12,2017,12
18062,198348,000,2017,12,2017,12
18063,198392,000,2017,12,2017,12
18064,198472,000,2017,12,2017,12


In [60]:
result_df["prefix"] = result_df["prefix"].astype(str)
result_df["suffix"] = result_df["suffix"].astype(str)
result_df.to_csv("prefix_suffix_result.csv")

In [47]:
df_us_firms_specific[df_us_firms_specific["Company Mapping No."] == "22043001"].sort_values(by=["Year", "Month"])

,Company Mapping No.,Year,Month
29,22043001,2004,9
30,22043001,2004,10
31,22043001,2004,11
32,22043001,2004,12
33,22043001,2005,1
...,...,...,...
127,22043001,2012,11
128,22043001,2012,12
129,22043001,2013,1
130,22043001,2013,2


In [65]:
result_df

,prefix,suffix,start_year,start_month,end_year,end_month
0,22043,000,2015,8,2017,12
1,22043,001,2004,9,2013,3
2,26978,000,2009,5,2017,12
3,26978,001,2006,5,2009,4
4,26978,002,2005,2,2006,4
...,...,...,...,...,...,...
18061,198308,000,2017,12,2017,12
18062,198348,000,2017,12,2017,12
18063,198392,000,2017,12,2017,12
18064,198472,000,2017,12,2017,12


In [66]:
result_df[result_df['prefix'] == '26978']

,prefix,suffix,start_year,start_month,end_year,end_month
2,26978,000,2009,5,2017,12
3,26978,001,2006,5,2009,4
4,26978,002,2005,2,2006,4
5,26978,003,2003,8,2005,1
6,26978,004,1988,1,2003,7
